# End to End ML with Metaflow and Tempo

We will train two models and deploy them with tempo within a Metaflow pipeline. To understand the core example see [here](https://tempo.readthedocs.io/en/latest/examples/multi-model/README.html)

![archtecture](architecture.png)

## MetaFlow Prequisites


### Install metaflow locally

```
pip install metaflow
```

### Setup Conda-Forge Support

The flow will use conda-forge so you need to add that channel to conda.

```
conda config --add channels conda-forge
```



## Iris Flow Summary

In [1]:
!python src/irisflow.py --environment=conda show

Metaflow 2.3.2 executing IrisFlow for user:clive

A Flow to train two Iris dataset models and combine them for inference with Tempo

The flow performs the following steps:

1) Load Iris Data
2) Train SKLearn LR Model
3) Train XGBoost LR Model
4) Create and deploy Tempo artifacts

Step start
    Download Iris classification datatset
    => train_sklearn, train_xgboost

Step train_sklearn
    Train a SKLearn Logistic Regression Classifier on dataset and save model as artifact
    => join

Step train_xgboost
    Train an XGBoost classifier on the dataset and save model as artifact
    => join

Step join
    Merge two training runs.
    => tempo

Step tempo
    Create Tempo artifacts locally and saved to S3 within the workflow bucket.
    Then either deploy locally to Docker or deploy to a remote Kubernetes cluster based on the
    --tempo-on-docker parameter
    => end

Step end
    End flow.



## Run Flow locally to deploy to Docker

To run the workflow with a local Docker deployment use the flag:

```
--tempo-on-docker true
```


In [2]:
!python src/irisflow.py --environment=conda run 

Metaflow 2.3.2 executing IrisFlow for user:clive
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
Bootstrapping conda environment...(this could take a few minutes)
Including file src/conda.yaml of size 115B 
File persisted at s3://metaflow1-metaflows3bucket-1ou61547mcwbq/metaflow/data/IrisFlow/a94f1cff7702ed70807d16917bb282f51a28511e
Including file src/gsa-key.json of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-1ou61547mcwbq/metaflow/data/IrisFlow/c39c6e4ed18f183a2a3ae16b491f2848b57f6824
Including file src/kubeconfig.yaml of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-1ou61547mcwbq/metaflow/data/IrisFlow/a203db53bd80aea425e9fdc206db3d107ce77f69
2021-09-04 09:15:26.734 Workflow starting (run-id 2):
2021-09-04 09:15:28.219 [2/start/9 (pid 535320)] Task is starting.
2021-09-04 09:15:32.460 [2/start/9 (pid 535320)] Task finished successfully.
2021-09-04 09:15:34.087 [2/train_sklearn/10 (p

2021-09-04 08:17:28.850 [2/tempo/13 (pid 535666)] Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
2021-09-04 08:17:28.850 [2/tempo/13 (pid 535666)] Collecting cachetools<5.0,>=2.0.0
2021-09-04 08:17:28.851 [2/tempo/13 (pid 535666)] Using cached cachetools-4.2.2-py3-none-any.whl (11 kB)
2021-09-04 08:17:28.851 [2/tempo/13 (pid 535666)] Collecting pyasn1-modules>=0.2.1
2021-09-04 08:17:28.851 [2/tempo/13 (pid 535666)] Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
2021-09-04 08:17:28.853 [2/tempo/13 (pid 535666)] Collecting rsa<5,>=3.1.4
2021-09-04 08:17:28.853 [2/tempo/13 (pid 535666)] Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
2021-09-04 08:17:28.853 [2/tempo/13 (pid 535666)] Collecting pyasn1<0.5.0,>=0.4.6
2021-09-04 08:17:28.853 [2/tempo/13 (pid 535666)] Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
2021-09-04 08:17:28.853 [2/tempo/13 (pid 535666)] Collecting pyparsing>=2.0.2
2021-09-04 08:17:28.854 [2/tempo/13 (pid 535666)] Using cach

2021-09-04 08:17:46.002 [2/tempo/13 (pid 535666)] Preparing transaction: ...working... done
2021-09-04 08:17:46.151 [2/tempo/13 (pid 535666)] Verifying transaction: ...working... done
2021-09-04 08:17:46.326 [2/tempo/13 (pid 535666)] Executing transaction: ...working... done
2021-09-04 08:18:24.545 [2/tempo/13 (pid 535666)] Remove all packages in environment /home/clive/anaconda3/envs/tempo-bfe7ae29-ca47-4b67-a749-8c99f97d780e:
2021-09-04 08:18:24.545 [2/tempo/13 (pid 535666)] 
2021-09-04 08:18:24.546 [2/tempo/13 (pid 535666)] Insights Manager not initialised as empty URL provided.
2021-09-04 08:18:34.767 [2/tempo/13 (pid 535666)] {'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32), 'output1': 'xgboost prediction'}
2021-09-04 09:18:37.183 [2/tempo/13 (pid 535666)] Task finished successfully.
2021-09-04 09:18:39.206 [2/end/14 (pid 537519)] Task is starting.
2021-09-04 09:18:42.875 [2/end/14 (pid 537519)] Task finished successfully.
2021-09-04 09:18:43.323 Done!


## Make Predictions with Metaflow Tempo Artifact

In [3]:
from metaflow import Flow
import numpy as np
run = Flow('IrisFlow').latest_run
client = run.data.client_model
client.predict(np.array([[1, 2, 3, 4]]))

{'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32),
 'output1': 'xgboost prediction'}

## Run Flow on AWS and Deploy to Remote Kubernetes

We will now run our flow on AWS Batch and will launch Tempo artifacts onto a remote Kubernetes cluster. 

### Setup AWS Metaflow Support

Note at present this is required even for a local run as artifacts are stored on S3.

[Install Metaflow with remote AWS support](https://docs.metaflow.org/metaflow-on-aws/metaflow-on-aws).

### Seldon Requirements

For deploying to a remote Kubernetes cluster with Seldon Core installed do the following steps:

#### Install Seldon Core on your Kubernetes Cluster

Create a GKE cluster and install Seldon Core on it using [Ansible to install Seldon Core on a Kubernetes cluster](https://github.com/SeldonIO/ansible-k8s-collection).


### K8S Auth from Metaflow

To deploy services to our Kubernetes cluster with Seldon Core installed, Metaflow steps that run on AWS Batch and use tempo will need to be able to access K8S API. This step will depend on whether you're using GKE or AWS EKS to run 
your cluster.

#### Option 1. K8S cluster runs on GKE

We will need to create two files in the flow src folder:

```bash
kubeconfig.yaml
gsa-key.json
```

Follow the steps outlined in [GKE server authentication](https://cloud.google.com/kubernetes-engine/docs/how-to/api-server-authentication#environments-without-gcloud).




#### Option 2. K8S cluster runs on AWS EKS

Make note of two AWS IAM role names, for example find them in the IAM console. The names depend on how you deployed Metaflow and EKS in the first place:

1. The role used by Metaflow tasks executed on AWS Batch. If you used the default CloudFormation template to deploy Metaflow, it is the role that has `*BatchS3TaskRole*` in its name.

2. The role used by EKS nodes. If you used `eksctl` to create your EKS cluster, it is the role that starts with `eksctl-<your-cluster-name>-NodeInstanceRole-*`

Now, we need to make sure that AWS Batch role has permissions to access the K8S cluster. For this, add a policy to the AWS Batch task role(1) that has `eks:*` permissions on your EKS cluster (TODO: narrow this down).

You'll also need to add a mapping for that role to `aws-auth` ConfigMap in `kube-system` namespace. For more details, see [AWS docs](https://docs.aws.amazon.com/eks/latest/userguide/add-user-role.html) (under "To add an IAM user or role to an Amazon EKS cluster"). In short, you'd need to add this to `mapRoles` section in the aws-auth ConfigMap:
```
     - rolearn: <batch task role ARN>
       username: cluster-admin
       groups:
         - system:masters
```

We also need to make sure that the code running in K8S can access S3. For this, add a policy to the EKS node role (2) to allow it to read and write Metaflow S3 buckets.

### S3 Authentication
Services deployed to Seldon will need to access Metaflow S3 bucket to download trained models. The exact configuration will depend on whether you're using GKE or AWS EKS to run your cluster.

From the base templates provided below, create your `k8s/s3_secret.yaml`.

```yaml
apiVersion: v1
kind: Secret
metadata:
  name: s3-secret
type: Opaque
stringData:
  RCLONE_CONFIG_S3_TYPE: s3
  RCLONE_CONFIG_S3_PROVIDER: aws
  RCLONE_CONFIG_S3_BUCKET_REGION: <region>
  <...cloud-dependent s3 auth settings (see below)>
```

For GKE, to access S3 we'll need to add the following variables to use key/secret auth:
```yaml
  RCLONE_CONFIG_S3_ENV_AUTH: "false"
  RCLONE_CONFIG_S3_ACCESS_KEY_ID: <key>
  RCLONE_CONFIG_S3_SECRET_ACCESS_KEY: <secret>
```

For AWS EKS, we'll use the instance role assigned to the node, we'll only need to set one env variable:
```yaml
RCLONE_CONFIG_S3_ENV_AUTH: "true"
```

We provide two templates to use in the `k8s` folder:

```
s3_secret.yaml.tmpl.aws
s3_secret.yaml.tmpl.gke
```

Use one to create the file `s3_secret.yaml` in the same folder


## Setup RBAC and Secret on Kubernetes Cluster

These steps assume you have authenticated to your cluster with kubectl configuration

In [4]:
!kubectl create ns production

Error from server (AlreadyExists): namespaces "production" already exists


In [5]:
!kubectl create -f k8s/tempo-pipeline-rbac.yaml -n production

Error from server (AlreadyExists): error when creating "k8s/tempo-pipeline-rbac.yaml": serviceaccounts "tempo-pipeline" already exists
Error from server (AlreadyExists): error when creating "k8s/tempo-pipeline-rbac.yaml": roles.rbac.authorization.k8s.io "tempo-pipeline" already exists
Error from server (AlreadyExists): error when creating "k8s/tempo-pipeline-rbac.yaml": rolebindings.rbac.authorization.k8s.io "tempo-pipeline-rolebinding" already exists


Create a Secret from the `k8s/s3_secret.yaml.tmpl` file by adding your AWS Key that can read from S3 and saving as `k8s/s3_secret.yaml`

In [6]:
!kubectl create -f k8s/s3_secret.yaml -n production

Error from server (AlreadyExists): error when creating "k8s/s3_secret.yaml": secrets "s3-secret" already exists


## Run Metaflow on AWS Batch

In [8]:
!python src/irisflow.py \
    --environment=conda \
    --with batch:image=seldonio/seldon-core-s2i-python37-ubi8:1.10.0-dev \
    run

Metaflow 2.3.2 executing IrisFlow for user:clive
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
Bootstrapping conda environment...(this could take a few minutes)
Including file src/conda.yaml of size 115B 
File persisted at s3://metaflow1-metaflows3bucket-1ou61547mcwbq/metaflow/data/IrisFlow/a94f1cff7702ed70807d16917bb282f51a28511e
Including file src/gsa-key.json of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-1ou61547mcwbq/metaflow/data/IrisFlow/c39c6e4ed18f183a2a3ae16b491f2848b57f6824
Including file src/kubeconfig.yaml of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-1ou61547mcwbq/metaflow/data/IrisFlow/a203db53bd80aea425e9fdc206db3d107ce77f69
2021-09-04 10:20:17.745 Workflow starting (run-id 4):
2021-09-04 10:20:19.310 [4/start/22 (pid 557187)] Task is starting.
2021-09-04 09:20:20.457 [4/start/22 (pid 557187)] [ff14310f-71b1-499d-a1c9-40e18f7d1f9a] Task is starting (status SUBMITTE

2021-09-04 09:22:47.906 [4/join/25 (pid 558394)] [1b45a3e8-6eec-428e-bf48-9e0ce995b1dc] Bootstrapping environment...
2021-09-04 09:23:04.216 [4/join/25 (pid 558394)] [1b45a3e8-6eec-428e-bf48-9e0ce995b1dc] Environment bootstrapped.
2021-09-04 09:23:12.402 [4/join/25 (pid 558394)] [1b45a3e8-6eec-428e-bf48-9e0ce995b1dc] Task finished with exit code 0.
2021-09-04 10:23:13.349 [4/join/25 (pid 558394)] Task finished successfully.
2021-09-04 10:23:15.268 [4/tempo/26 (pid 558822)] Task is starting.
2021-09-04 09:23:16.485 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d] Task is starting (status SUBMITTED)...
2021-09-04 09:23:19.778 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d] Task is starting (status RUNNABLE)...
2021-09-04 09:23:20.892 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d] Task is starting (status STARTING)...
2021-09-04 09:23:23.097 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d] Task is starting (status RUNNING)

2021-09-04 09:24:40.470 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d] Collecting charset-normalizer~=2.0.0
2021-09-04 09:24:40.470 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d]   Downloading charset_normalizer-2.0.4-py3-none-any.whl (36 kB)
2021-09-04 09:24:40.470 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d] Collecting urllib3<1.27,>=1.21.1
2021-09-04 09:24:40.470 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d]   Downloading urllib3-1.26.6-py2.py3-none-any.whl (138 kB)
2021-09-04 09:24:40.470 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d] Requirement already satisfied: certifi>=2017.4.17 in /opt/conda/envs/tempo-bc2369b7-c961-49a5-9c58-ea225c950074/lib/python3.7/site-packages (from requests->mlops-tempo->-r /tmp/condaenv.tof6b1t3.requirements.txt (line 1)) (2021.5.30)
2021-09-04 09:24:40.470 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d] Collecting starlette==0.14.2
2021-09-04

2021-09-04 09:24:52.835 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d] The following packages will be REMOVED:
2021-09-04 09:24:52.835 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d]
2021-09-04 09:24:52.835 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d]   _libgcc_mutex-0.1-main
2021-09-04 09:24:52.835 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d]   _openmp_mutex-4.5-1_gnu
2021-09-04 09:24:52.835 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d]   ca-certificates-2021.7.5-h06a4308_1
2021-09-04 09:24:52.835 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d]   certifi-2021.5.30-py37h06a4308_0
2021-09-04 09:24:52.835 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d]   ld_impl_linux-64-2.35.1-h7274673_9
2021-09-04 09:24:52.835 [4/tempo/26 (pid 558822)] [31103662-b002-45a2-bd37-eaf23263f12d]   libffi-3.3-he6710b0_2
2021-09-04 09:24:52.835 [4/tempo/26 (pid 558822)] [31103662-b00

## Make Predictions with Metaflow Tempo Artifact

In [9]:
from metaflow import Flow
run = Flow('IrisFlow').latest_run
client = run.data.client_model
import numpy as np
client.predict(np.array([[1, 2, 3, 4]]))

{'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32),
 'output1': 'xgboost prediction'}